In [60]:
import pandas as pd
import numpy as np

In [6]:
%store -r final_merge

In [7]:
final_merge.head()

,order_id,order_date,state,region,ship_mode,ready_to_ship_date,pickup_date,arrival_scan_date
0,CA-2017-100006,2017-09-07,New York,East,Standard Class,NaT,NaT,NaT
1,CA-2017-100090,2017-07-08,California,West,Standard Class,NaT,NaT,NaT
2,CA-2017-100293,2017-03-14,Florida,South,Standard Class,NaT,NaT,NaT
3,CA-2017-100328,2017-01-28,New York,East,Standard Class,NaT,NaT,NaT
4,CA-2017-100363,2017-04-08,Arizona,West,Standard Class,NaT,NaT,NaT


In [45]:
order_start_day = final_merge.copy()

In [47]:
order_start_day = order_start_day.filter(items=['order_date','ready_to_ship_date'])
order_start_day.head(2)

,order_date,ready_to_ship_date
0,2017-09-07,NaT
1,2017-07-08,NaT


In [50]:
order_start_day.shape

(5010, 2)

In [48]:
# assess how many null values are in a dataframe with pandas
order_start_day.isnull().sum()

order_date               0
ready_to_ship_date    4806
dtype: int64

In [52]:
# dropping the null values
order_start_day.dropna(subset=['ready_to_ship_date'])

,order_date,ready_to_ship_date
1703,2019-09-20,2019-09-24
1711,2019-09-24,2019-09-30
1740,2019-09-11,2019-09-16
1747,2019-09-19,2019-09-23
1748,2019-09-06,2019-09-09
...,...,...
4985,2020-11-02,2020-11-04
4988,2020-11-13,2020-11-16
4990,2020-11-30,2020-12-03
5002,2020-11-04,2020-11-04


In [56]:
# create a new column based of the date column with the day of the week
order_start_day['order_day'] = order_start_day['order_date'].dt.dayofweek
order_start_day.head(2)

,order_date,ready_to_ship_date,order_day,order_day_name
0,2017-09-07,NaT,3,Thursday
1,2017-07-08,NaT,5,Saturday


In [54]:
# create a new column based of the date column with the day of the week
order_start_day['order_day_name'] = order_start_day['order_date'].dt.day_name()

order_start_day.head(2)

,order_date,ready_to_ship_date,order_day,order_day_name
0,2017-09-07,NaT,3,Thursday
1,2017-07-08,NaT,5,Saturday


In [67]:
# Calculate the processing date
order_start_day['order_processing'] = np.where(order_start_day['order_date'].dt.dayofweek == 5,
                                  order_start_day['order_date'] + pd.Timedelta(days=2),               
                                  order_start_day['order_date'])       
order_start_day['order_processing'] = np.where(order_start_day['order_date'].dt.dayofweek == 6,
                                  order_start_day['order_date'] + pd.Timedelta(days=1),               
                                  order_start_day['order_date'])                                
order_start_day



,order_date,ready_to_ship_date,order_day,order_day_name,order_processing
0,2017-09-07,NaT,3,Thursday,2017-09-07
1,2017-07-08,NaT,5,Saturday,2017-07-08
2,2017-03-14,NaT,1,Tuesday,2017-03-14
3,2017-01-28,NaT,5,Saturday,2017-01-28
4,2017-04-08,NaT,5,Saturday,2017-04-08
...,...,...,...,...,...
5005,2020-11-03,2020-11-09,1,Tuesday,2020-11-03
5006,2020-07-23,NaT,3,Thursday,2020-07-23
5007,2020-09-07,NaT,0,Monday,2020-09-07
5008,2020-08-28,NaT,4,Friday,2020-08-28


In [72]:
for index, row in order_start_day.iterrows():
    day_of_week = row['order_date'].dayofweek
    if day_of_week == 5:  # Saturday
        order_start_day.loc[index, 'order_processing'] = row['order_date'] + pd.Timedelta(days=2)
    elif day_of_week == 6:  # Sunday
        order_start_day.loc[index, 'order_processing'] = row['order_date'] + pd.Timedelta(days=1)
    else:
        order_start_day.loc[index, 'order_processing'] = row['order_date']
order_start_day

,order_date,ready_to_ship_date,order_day,order_day_name,order_processing
0,2017-09-07,NaT,3,Thursday,2017-09-07
1,2017-07-08,NaT,5,Saturday,2017-07-10
2,2017-03-14,NaT,1,Tuesday,2017-03-14
3,2017-01-28,NaT,5,Saturday,2017-01-30
4,2017-04-08,NaT,5,Saturday,2017-04-10
...,...,...,...,...,...
5005,2020-11-03,2020-11-09,1,Tuesday,2020-11-03
5006,2020-07-23,NaT,3,Thursday,2020-07-23
5007,2020-09-07,NaT,0,Monday,2020-09-07
5008,2020-08-28,NaT,4,Friday,2020-08-28


In [70]:
# Filter rows where the day of the week is Sunday
sunday_orders = order_start_day[order_start_day['order_date'].dt.dayofweek == 6]  # 6 represents Sunday
sunday_orders

,order_date,ready_to_ship_date,order_day,order_day_name,order_processing
1,2017-07-08,NaT,5,Saturday,2017-07-08
3,2017-01-28,NaT,5,Saturday,2017-01-28
4,2017-04-08,NaT,5,Saturday,2017-04-08
7,2017-12-16,NaT,5,Saturday,2017-12-16
9,2017-03-26,NaT,6,Sunday,2017-03-27
...,...,...,...,...,...
4966,2020-01-12,NaT,6,Sunday,2020-01-13
4987,2020-02-02,NaT,6,Sunday,2020-02-03
4989,2020-07-18,NaT,5,Saturday,2020-07-18
4996,2020-03-28,NaT,5,Saturday,2020-03-28


In [64]:
# Calculate the processing date
order_start_day['order_processing'] = np.where(order_start_day['order_date'].dt.dayofweek >= 4,       # Friday, Saturday, or Sunday
                                  order_start_day['order_date'] + pd.Timedelta(days=3),               # Add 3 days to move to Monday
                                  order_start_day['order_date'])                                      # Otherwise keep the order_date
order_start_day


,order_date,ready_to_ship_date,order_day,order_day_name,order_processing
0,2017-09-07,NaT,3,Thursday,2017-09-07
1,2017-07-08,NaT,5,Saturday,2017-07-11
2,2017-03-14,NaT,1,Tuesday,2017-03-14
3,2017-01-28,NaT,5,Saturday,2017-01-31
4,2017-04-08,NaT,5,Saturday,2017-04-11
...,...,...,...,...,...
5005,2020-11-03,2020-11-09,1,Tuesday,2020-11-03
5006,2020-07-23,NaT,3,Thursday,2020-07-23
5007,2020-09-07,NaT,0,Monday,2020-09-07
5008,2020-08-28,NaT,4,Friday,2020-08-31
